## Le seuillage d'images (Images thresholding)

### <span style=color:green> Seuillage d'une en image en niveau de gris </span>

La segmentation par seuillage est une binarisation de l'image. Elle divise donc l'image en deux classes et est appliquée sur une image en niveau de gris.

Pour choisir le seuil de segmentation, une analyse préable de l'histogramme et/ou des statisques sur l'image est nécessaire afin de répérer les différents modes. Le choix du seuil peut être difficile dans certains cas, on peut alors tester plusieurs valeurs ou faire une observation sur l'image pour répérer les tons de gris sombres et clairs.

1. **le seuillage global** : le choix du seuil est facile car l'histogramme présente deux modes assez disctincts.
2. **le seuillage automatique d'Otsu** : L'image ne contient que deux classes (avant-plan et arrière-plan) puis l'algorithme calcule le seuil optimal en minimisant leur variance pondérée intra-classe. Autrement dit, Otsu cherche le seuil qui rend les pixels de chaque classe aussi similaires que possible. Pour chaque classe, la variance intra-classe mesure à quel point les valeurs de gris des pixels de cette classe sont étroitement regroupées autour de la moyenne de cette classe.
3. **le seuillage multiple** : 3 modes ou plus distincts sont visibles sur l'histogramme, on a plus de deux classes. on peut utiliser l'algorithme multi Otsu.
4. **le seuillage adaptatif** : efficace pour ombrage et éclairage non uniforme.

### <span style=color:green> Seuillage d'image RGB </span>

Le seuillage se fait ici sur les différents ou sur l'un des canaux. L'espace RGB mélange à la fois la luminance et la chrominance donc un seuillage sur un canal ou plus altèrera la couleur. Il est alors préférable de travailler sur l'espace HSV ou YUV en travaillant sur le canal V.